In [1]:
import xarray as xr
import rioxarray

In [2]:
data_path = "/mnt/CEPH_PROJECTS/InterTwin/hydrologic_data/cerra/2m_temperature/t2m_2001.nc"
ds = xr.open_dataset(data_path)
ds.rio.write_crs("epsg:4326", inplace=True)

ds.to_netcdf("/mnt/CEPH_PROJECTS/InterTwin/hydrologic_data/cerra/2m_temperature/t2m_2001_w_crs.nc")

In [3]:
ds

<xarray.Dataset>
Dimensions:      (latitude: 98, longitude: 163, time: 365)
Coordinates:
  * latitude     (latitude) float64 43.55 43.62 43.69 43.75 ... 49.93 50.0 50.06
  * longitude    (longitude) float64 5.084 5.151 5.218 ... 15.82 15.89 15.96
  * time         (time) datetime64[ns] 2001-01-01 2001-01-02 ... 2001-12-31
    spatial_ref  int64 0
Data variables:
    t2m          (time, latitude, longitude) float32 ...

In [4]:
import sys
sys.path.append("/home/rbalogun/intertwin/raster-to-stac")
from raster2stac import raster2stac as r2slib

r2s = r2slib.Raster2STAC(
    "/mnt/CEPH_PROJECTS/InterTwin/hydrologic_data/cerra/2m_temperature/t2m_2001_w_crs.nc", # data
    "CERRA_2m_temperature_2001", # collection_id
    "https://stac.eurac.edu/collection", # collection_url
    output_folder="/home/rbalogun/intertwin/cerra/stac/",
    output_file='CERRA_2m_temperature_2001.json',
    description="CERRA 2m_temperature_2001 collection for InterTwin",
    title="CERRA_2m_temperature_2001",
    ignore_warns=False,
    keywords=['intertwin', 'temperature', 'cerra', 'climate'],
    links= [{
        "rel": "license",
        "href": "https://cds.climate.copernicus.eu/api/v2/terms/static/licence-to-use-copernicus-products.pdf",
        "title": "License to use Copernicus Products"
    }],
    providers=[
        {
            "url": "https://cds.climate.copernicus.eu/cdsapp#!/dataset/10.24381/cds.622a565a",
            "name": "Copernicus",
            "roles": [
                "producer",
                "licensor"
            ]
        },
        {
            "url": "http://www.eurac.edu",
            "name": "Eurac EO",
            "roles": [
            "host"
            ]
        }
    ],
    stac_version="1.0.0",
    verbose=True,
    s3_upload=False,
    version=None,
    output_format="csv",
    license="proprietary",
    write_json_items=True,
    sci_doi='https://doi.org/10.24381/cds.622a565a',
    sci_citation= "Schimanke S., Ridal M., Le Moigne P., Berggren L., Undén P., Randriamampianina R., Andrea U., \
        Bazile E., Bertelsen A., Brousseau P., Dahlgren P., Edvinsson L., El Said A., Glinton M., Hopsch S., \
        Isaksson L., Mladek R., Olsson E., Verrelle A., Wang Z.Q., (2021): CERRA sub-daily regional reanalysis \
        data for Europe on single levels from 1984 to present. Copernicus Climate Change Service (C3S) Climate \
        Data Store (CDS), DOI: 10.24381/cds.622a565a (Accessed on 15-02-2024)"
)

r2s.generate_stac()
